In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feedback or issues?

For any feedback or questions, please open an [issue](https://github.com/googleapis/python-aiplatform/issues).

# Vertex SDK for Python: AutoML Text Extraction Example
To use this Jupyter notebook, copy the notebook to a Google Cloud Notebooks instance with Tensorflow installed and open it. You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Jupyter automatically displays the return value of the last line in each cell. For more information about running notebooks in Google Cloud Notebook, see the [Google Cloud Notebook guide](https://cloud.google.com/vertex-ai/docs/general/notebooks).


This notebook demonstrate how to create an AutoML Text Extraction Model, with a Vertex AI text dataset, and how to serve the model for online prediction.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

### Install Vertex SDK for Python


After the SDK installation the kernel will be automatically restarted.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Enter Your Project and GCS Bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [ ]:
MY_PROJECT = "YOUR PROJECT ID"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as ucaip

## Initialize Vertex SDK for Python

Initialize the *client* for Vertex AI.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

## Create a Dataset on Vertex AI
We will now create a Vertex AI text dataset using the previously prepared jsonl files. 

### The biomedical dataset
To create an entity extraction model, use a corpus of biomedical research abstracts that mention hundreds of diseases and concepts. The resulting model identifies these medical entities in other documents.

The goal of the corpus is to advance the understanding of the causes of happiness through text-based reflection.

Please reference [AutoML Documentation](https://cloud.google.com/natural-language/automl/docs/quickstart#model_objectives) for more information.

In [ ]:
# Text Extraction
IMPORT_FILE = "gs://ucaip-test-us-central1/dataset/ucaip_ten_dataset.jsonl"

In [ ]:
ds = aiplatform.TextDataset.create(
    display_name="text-extraction",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.extraction,
)

ds.resource_name

## Launch a Training Job and Create a Model on Vertex AI

In [ ]:
job = aiplatform.AutoMLTextTrainingJob(
    display_name="text-extraction", prediction_type="extraction"
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    model_display_name="text-extraction",
)

# Deploy Model

In [ ]:
endpoint = model.deploy()

# Predict on Endpoint

In [ ]:
input_text = """
Phenotypic variation including retinitis pigmentosa, pattern dystrophy, and fundus flavimaculatus in a single family with a deletion of codon 153 or 154 of the peripherin/RDS gene.\tBACKGROUND AND OBJECTIVES  Mutations of the peripherin / RDS gene have been reported in autosomal dominant retinitis pigmentosa , pattern macular dystrophy , and retinitis punctata albescens . We report herein the occurrence of three separate phenotypes within a single family with a novel 3-base pair deletion of codon 153 or 154 of the peripherin / RDS gene . DESIGN  Case reports with clinical features , fluorescein angiography , kinetic perimetry , electrophysiological studies , and molecular genetics . SETTING  University medical centers . PATIENTS  A 75-year-old woman , her two daughters ( aged 44 and 50 years ) , and her 49-year-old son were screened for peripherin / RDS mutations because of the presence of multiple phenotypes within the same family . RESULTS  The mother presented at age 63 years with a profoundly abnormal electroretinogram ( ERG ) and adult-onset retinitis pigmentosa that progressed dramatically over 12 years , with marked loss of peripheral visual field . One daughter developed pattern macular dystrophy at age 31 years . At age 44 years , her ERG was moderately abnormal but her clinical disease was limited to the macula . Another daughter presented at age 42 years with macular degeneration and over 10 years developed the clinical picture of fundus flavimaculatus . Her peripheral visual field was preserved but her ERG was moderately abnormal . The son had onset of macular degeneration at age 44 years . Pericentral scotomas were present and the ERG was markedly abnormal . Fluorescein angiography revealed punctate pigment epithelial transmission defects . CONCLUSIONS  A 3-base pair deletion of codon 153 or 154 of the peripherin / RDS gene can produce clinically disparate phenotypes even within the same family
Splicing defects in the ataxia-telangiectasia gene, ATM: underlying mutations and consequences.\tMutations resulting in defective splicing constitute a significant proportion ( 30 / 62 [ 48 % ] ) of a new series of mutations in the ATM gene in patients with ataxia-telangiectasia ( AT ) that were detected by the protein-truncation assay followed by sequence analysis of genomic DNA . Fewer than half of the splicing mutations involved the canonical AG splice-acceptor site or GT splice-donor site . A higher percentage of mutations occurred at less stringently conserved sites , including silent mutations of the last nucleotide of exons , mutations in nucleotides other than the conserved AG and GT in the consensus splice sites , and creation of splice-acceptor or splice-donor sites in either introns or exons . These splicing mutations led to a variety of consequences , including exon skipping and , to a lesser degree , intron retention , activation of cryptic splice sites , or creation of new splice sites . In addition , 5 of 12 nonsense mutations and 1 missense mutation were associated with deletion in the cDNA of the exons in which the mutations occurred . No ATM protein was detected by western blotting in any AT cell line in which splicing mutations were identified . Several cases of exon skipping in both normal controls and patients for whom no underlying defect could be found in genomic DNA were also observed , suggesting caution in the interpretation of exon deletions observed in ATM cDNA when there is no accompanying identification of genomic mutations .
"""

instances_list = [{"content": input_text}]

prediction = endpoint.predict(instances_list)
prediction

In [ ]:
prediction_instance = prediction.predictions[0]

extractions = zip(
    prediction_instance["ids"],
    prediction_instance["textSegmentStartOffsets"],
    prediction_instance["textSegmentEndOffsets"],
    prediction_instance["confidences"],
    prediction_instance["displayNames"],
)

for id, start, end, confidence, display_name in extractions:
    print(
        f"{id}: '{input_text[int(start):int(end)]}' predicted as '{display_name}'' with confidence {confidence}"
    )